In [1]:
%config Completer.use_jedi = False

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

## ToDo

In [2]:
# basic conditional gan
# generate data
# add and train a classifier use k fold also
# condition me one hot encoding
# generator k lie combined data and disc k lie train data.. coz testing me sara deke bias ho jaega
# different loss functions for generator and discriminator
# discriminator output shud be one hot encode with labels as n s or v and not 0 or 1 that is fake or real:
# confusion here
# disc property: should recognise fake and also if gen data is not of that class (so 2 things)
# training k time jab random samples uthaenge tab sari classes k
# equal no. of samples uthana

# refer manning book
# For example, the CGAN Discriminator should learn to reject the pair (, 4), regardless of whether the 
# example (handwritten numeral 3) is real or fake, because it does not match the label, 4. 
# The CGAN Discriminator should also learn to reject all image-label pairs in which the image is fake, 
# even if the label matches the image.

# Accordingly, in order to fool the Discriminator, it is not enough for the CGAN Generator to produce 
# realistic-looking data. The examples it generates also need to match their labels. 
# After the Generator is fully trained, this then allows us to specify what example we want the CGAN to 
# synthesize by passing it the desired label.

## Import and Param Define

In [3]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from helper import *
import keras as keras
from pycm import *
from sklearn.metrics import classification_report, confusion_matrix

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import plot_model

# Data 

In [4]:
# X_test = np.load('Data/test_data.npy')
# X_train = np.load('Data/train_data.npy')

# y_train = X_train[:,-1]
# X_train = X_train[:,:-1]

# y_test = X_test[:,-1]
# X_test = X_test[:,:-1]

# X_Gen = np.vstack((X_train, X_test))
# y_Gen = np.hstack((y_train, y_test))

# np.save('X_Gen', X_Gen)
# np.save('y_Gen', y_Gen)

# X_Gen = np.load('Data/X_Gen.npy')
# y_Gen = np.load('Data/y_Gen.npy')

# print (X_Gen.shape, y_Gen.shape)

# X_N = X_Gen[y_Gen==0][:80000]
# X_S = X_Gen[y_Gen==1]
# X_V = X_Gen[y_Gen==2]

# y_N = y_Gen[y_Gen==0][:80000]
# y_S = y_Gen[y_Gen==1]
# y_V = y_Gen[y_Gen==2]

# X = np.vstack((X_N, X_S, X_V))
# y = np.hstack((y_N, y_S, y_V))

# print (X.shape, y.shape)

# np.save('Data/X', X)
# np.save('Data/y', y)

# print (y_test.shape, X_test.shape, y_train.shape, X_train.shape, X_Gen.shape, y_Gen.shape)

X = np.load('Data/X.npy')
y = np.load('Data/y.npy')

print (X.shape, y.shape)
# X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
# X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)
# y_train = keras.utils.to_categorical(y_train)
# y_test = keras.utils.to_categorical(y_test)
# X_Gen=X_Gen.reshape(X_Gen.shape[0],X_Gen.shape[1],1)
# y_Gen = keras.utils.to_categorical(y_Gen)

# print (X_Gen.shape, y_Gen.shape)

(176212, 186) (176212,)


In [5]:
X.shape

(176212, 186)

In [6]:
y.shape

(176212,)

## Training 

In [ ]:
WINDOW_SIZE = 186
INPUT_FEAT = 1
OUTPUT_CLASS = 3

In [ ]:
for LAYER in range(1,8):
    filename = 'ResNet_' + str(LAYER)
    print (filename)
    if not os.path.exists(str(filename)):
        os.makedirs(str(filename))
    
    model = ResNet_model(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS,LAYER)
#     model.summary()

    history = model.fit(X_train, y_train, batch_size=32, epochs=1, callbacks=cb, validation_split=0.2, verbose=1)
    prediction(model, X_test, y_test, filename)
    save_the_model(filename, model)
    write_history(filename, history.history)

In [10]:
# output_directory = 'Parallel_NN'
# if not os.path.isdir(output_directory):
#     os.mkdir(output_directory)
# twelve_lead_model_filename = model_directory + '/' + twelve_lead_filename
# train_generator = generator(X_train, y_train, Window, leads, batch_size=BS)
# train_samples=np.ceil(y_train.shape[0] / BS)
# val_generator = generator(X_val, y_val, Window, leads, batch_size=BS)
# val_samples=np.ceil(y_val.shape[0] / BS)

callback = [EarlyStopping(monitor='val_AUC', mode='max', verbose=1, patience=Pat),
         ModelCheckpoint(filepath=str(twelve_lead_model_filename)+'_check_model.h5', 
                         monitor='val_AUC', verbose=1, save_best_only=True, mode='max'),
         ReduceLROnPlateau(monitor='val_AUC', factor=0.5, patience=Pat//2, verbose=1, 
                           mode='max', min_delta=0.0001, cooldown=0, min_lr=0)]

# model = parallel_NN(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS):
model = parallel_NN(Window, len(leads), snomed_classes.shape[0])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),
                       tf.keras.metrics.Recall(name='Recall'),
                       tf.keras.metrics.Precision(name='Precision'),
                       tf.keras.metrics.AUC(num_thresholds=200,summation_method="interpolation",
                                            name="AUC",dtype=None,curve="ROC",thresholds=None,
                                            multi_label=True,label_weights=None)])
history = model.fit(train_generator, steps_per_epoch=train_samples, epochs=EP, verbose=1,
                validation_data=val_generator, validation_steps=val_samples, callbacks=callback)


history_name = output_directory + '/' + twelve_lead_filename
print (twelve_lead_model_filename, history_name)

save_model(twelve_lead_model_filename, model)
write_history(history_name, history.history)

Epoch 1/40
 487/2425 [=====>........................] - ETA: 5:08 - loss: 0.1510 - accuracy: 0.9536 - Recall: 0.0951 - Precision: 0.2961 - AUC: 0.5546

KeyboardInterrupt: 